---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [17]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import re

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

gdp = (pd.read_excel('gdplev.xls', skiprows=219)
    .drop(['Unnamed: 0',
            'Unnamed: 1',
            'Unnamed: 2',
            'Unnamed: 3',
            9926.1,
            'Unnamed: 7'], axis=1)
    .dropna())
gdp.columns = ['YearQuarter', 'GDP in billions of chained 2009 dollars']
gdp = gdp.set_index('YearQuarter')
gdp['Percent Change'] = gdp['GDP in billions of chained 2009 dollars'].pct_change()

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    state_regions = []
    f = open('university_towns.txt', 'r')
    state = ''
    for line in f:
        m_state = re.search(r'(?P<state>[^(]+)\[edit\]', line)
        m_region = re.search(r'(?P<region>[^(]+:*)(\s$|( \(.+$))', line)
        if m_state:
            state = m_state.group('state')
        elif m_region: 
            state_regions.append([state, m_region.group('region')])
            
    f.close()
    return pd.DataFrame(state_regions, columns=["State", "RegionName"])

get_list_of_university_towns()

In [4]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''

    changes = gdp['Percent Change'].tolist()
    for i in range(1, len(changes)-1):
        if changes[i] < 0 and changes[i+1] < 0:
            return gdp.iloc[i].name

get_recession_start()

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    start = get_recession_start()
    changes = gdp['Percent Change'].tolist()
    for i in range(gdp.index.get_loc(start), len(changes)-1):
        if changes[i] > 0 and changes[i+1] > 0:
            return gdp.iloc[i+1].name

get_recession_end()

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    s_idx = gdp.index.get_loc(get_recession_start())
    e_idx = gdp.index.get_loc(get_recession_end())
    recession = gdp[s_idx:e_idx]['GDP in billions of chained 2009 dollars']
    return recession[recession == recession.min()].index[0]

get_recession_bottom()

In [7]:
def month_to_quarter(m):
    return m[0:4] + 'q' + str((int(m[-2:])-1)//3 + 1)

def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    houses = (pd.read_csv('City_Zhvi_AllHomes.csv'))
    houses['State'] = houses['State'].apply(lambda row: states[row])
    houses = (houses.set_index(['State', 'RegionName'])
            .ix[:, '2000-01':])
    return houses.T.groupby(month_to_quarter).mean().T
    
convert_housing_data_to_quarters()

In [63]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    start = get_recession_start()
    bottom = get_recession_bottom()
    rec_df = convert_housing_data_to_quarters().ix[:, start:bottom]
    rec_df['price_ratio'] = rec_df[start] - rec_df[bottom]
    uni_towns = get_list_of_university_towns().set_index(['State', 'RegionName'])
    uni_towns_house= pd.merge(uni_towns, rec_df, how='inner', left_index=True, right_index=True).dropna()
    notuni_towns_house= pd.merge(uni_towns, rec_df, how='outer', left_index=True, right_index=True).dropna()
    
    pvalue = ttest_ind(uni_towns_house['price_ratio'], notuni_towns_house['price_ratio']).pvalue
    better = 'university town' if uni_towns_house['price_ratio'].mean() < notuni_towns_house['price_ratio'].mean() \
        else 'non-university town'
    return (pvalue < 0.01, pvalue, better)
            
run_ttest()

(True, 0.0051533729302617667, 'university town')